In [ ]:
import functools
import numpy
import random

In [1]:
class Candidate:
    def __init__(self):
        self.crowding_distance = 0
        self.domination_count = 0
        self.rank = 0
        self.dominated_candidates = None
        self.features = None
        self.dominates = None


In [ ]:
class Problem:
    def __init__(self, objectives, bounds, dims, fitness):
        self.objectives = objectives
        self.bounds = bounds
        self.dims = dims
        self.n = len(bounds)
        self.fitness = fitness
        
    def __dominates(self, candidateA, candidateB):
        fA = self.fitness(candidateA)
        fB = self.fitness(candidateB)
        
        not_dominated = all(map(lambda f: f[0] <= f[1], zip(fA, fB)))
        dominates = any(map(lambda f: f[0] < f[1], zip(fA, fB)))
        
        return dominates and not_dominated
        
    def generate_candidate(self):
        candidate = Candidate()
        candidate.features = []
        candidate.dominates = functools.partial(self.__dominates,
                                                candidateA=candidate)
        
        for idx in range(self.n):
            candidate.features += random.uniform(self.bounds[idx][0], self.bounds[idx][1])
            
        return candidate


In [8]:
class NSGA2:
    def __init__(self, problem, pop_size, max_gen, cr, m, seed=42.0):
        self.pop_size = pop_size        # Population size
        self.max_gen = max_gen          # Max generations to evolve for
        self.cr = cr                    # Cross-over probability
        self.m = m                      # Mutation probability
        self.eps = 1e-3                 # Mutation strength
        self.seed = seed                # Random seed
        
        self.problem = problem          # Instance of the problem, ie. objective function/s
        
        self.population = None
        
        self.initialize()
        
    def initialize(self):
        self.population = []
        random.seed(self.seed)
        
        for idx in range(self.pop_size):
            self.population += self.problem.generate_candidate()
    
    def _mutate(self, candidate, n=1):
        for idx in range(len(candidate.features)):
            if random.random() > self.m:
                candidate.features[idx] += self.eps * (random.random() - 1) / 2
                if candidate.features[idx] > self.problem.bounds[idx][1]:
                    candidate.features[idx] = self.problem.bounds[idx][1]
                elif candidate.features[idx] < self.problem.bounds[idx][0]:
                    candidate.features[idx] = self.problem.bounds[idx][0]
                    
        return candidate
    
    def _crossover(self, candidateA, candidateB, n=1):
        for idx in range(len(candidateA.features)):
            if random.random() > self.cr:
                candidateA.features[idx], candidateB.features[idx]\
                = candidateB.features[idx], candidateA.features[idx]
                
        return candidateA, candidateB
        
    def _non_dominated_sort(self):
        fronts = [[]]
        
        for candidateA in self.population:
            candidateA.domination_count = 0
            candidateA.dominated_candidates = set()
            
            for candidateB in self.population:
                if candidateA.dominates(candidateB):
                    candidateA.dominated_candidates.add(candidateB)
                elif candidateB.dominates(candidateA):
                    candidateA.domination_count += 1
                    
            if candidateA.domination_count == 0:
                candidateA.rank = 0
                fronts[0].append(candidateA)
        
        idx = 0
        while len(fronts[idx]) > 0:
            ith_front = []
            
            for candidateA in fronts[idx]:
                for candidateB in candidateA.dominated_candidates:
                    candidateB.domination_count -= 1
                    
                    if candidateB.domination_count == 0:
                        candidateB.rank = i + 1
                        ith_front.append(candidateB)
                        
            i += 1
            fronts.append(ith_front)
            
        return fronts           
    
    def _crowding_distance(self, individual, front):
        pass
        
    def solve(self):
        pass
        

In [3]:
def f1(x):
    return x ** 2

def f2(x):
    return (x - 2) ** 2

def fitness(candidate):
    """minimise for both f1, f2"""
    return 1/(f1(candidate[0]) + f2(candidate[1]))

objectives = [f1, f2]  